# XGBoost Training for Municipal Time Series Forecasting

This notebook implements a comprehensive approach to training XGBoost models for each municipality (CD_MUN) in the dataset. The goal is to predict the sum of the next 4 weeks based on a 48-week lookback period.

## Overview
- **Data**: Time series data by municipality with weekly target values
- **Approach**: One XGBoost model per municipality using sliding window
- **Evaluation**: Comprehensive visualization and performance metrics

## Table of Contents
1. Setup and Library Imports
2. Define Run Identifier and Load Data
3. Data Preprocessing
4. Feature Engineering
5. Training Framework
6. Model Training Loop
7. Evaluation and Visualization
8. Results Summary

## Environment Setup

First, we need to set up the environment and install necessary dependencies. **IMPORTANT: After running this cell, restart the kernel before continuing.**

In [1]:
import sys
import subprocess

# Print current environment information
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print(f"Current environment: {sys.prefix}")

# Get the environment name
try:
    # Extract environment name from path
    import os
    env_path = sys.prefix
    env_name = os.path.basename(env_path)
    if env_name == '':  # If it's the base directory
        env_name = 'base'
    print(f"Detected environment name: {env_name}")
except Exception as e:
    print(f"Error detecting environment name: {e}")
    env_name = '.conda'  # Default fallback

# Method 1: Install using conda system call
print("\nMethod 1: Installing ipykernel using conda...")
try:
    result = subprocess.run(
        f"conda install -n {env_name} ipykernel --update-deps --force-reinstall -y",
        shell=True, check=True, capture_output=True, text=True
    )
    print("Conda install completed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Conda install error: {e}")
    print("Output:", e.output)
    print("Stderr:", e.stderr)

# Method 2: Install using pip
print("\nMethod 2: Installing with pip...")
try:
    !pip install --upgrade ipykernel
    print("Pip install completed.")
except Exception as e:
    print(f"Pip install error: {e}")

# Install other required packages
print("\nInstalling other required packages...")
!pip install pandas>=1.3 numpy>=1.21 matplotlib>=3.4 seaborn>=0.11 scikit-learn>=1.0 xgboost>=1.5 tqdm>=4.62 joblib>=1.1

# Try installing from requirements.txt if it exists
try:
    if os.path.exists("requirements.txt"):
        !pip install -r requirements.txt
        print("Installed packages from requirements.txt")
except Exception as e:
    print(f"Error installing from requirements.txt: {e}")

print("\n" + "="*50)
print("IMPORTANT: RESTART THE KERNEL BEFORE CONTINUING")
print("After restarting, the ipykernel will be properly installed.")
print("="*50)

Python version: 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)]
Python executable: c:\Users\pedro\OneDrive - Unesp\Documentos\GitHub\treinamento_clusters_hpc\.conda\python.exe
Current environment: c:\Users\pedro\OneDrive - Unesp\Documentos\GitHub\treinamento_clusters_hpc\.conda
Detected environment name: .conda

Method 1: Installing ipykernel using conda...
Conda install error: Command 'conda install -n .conda ipykernel --update-deps --force-reinstall -y' returned non-zero exit status 1.
Output: 
Stderr: 
EnvironmentLocationNotFound: Not a conda environment: C:\Users\pedro\.conda\envs\.conda



Method 2: Installing with pip...
Pip install completed.

Installing other required packages...
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11

# How to Restart the Kernel

After installing packages, you need to restart the kernel for the changes to take effect. Here's how to do it:

### Method 1: Using the Menu
- Click on the `Kernel` menu at the top of the notebook
- Select `Restart` or `Restart Kernel...`

### Method 2: Using Buttons
- Look for a restart button in the toolbar (often looks like a circular arrow ↻)
- In VS Code, it's in the top-right corner of the notebook window

### Method 3: Keyboard Shortcuts
- In Jupyter Notebook/Lab: Press `Ctrl+M` followed by `R` (Windows/Linux) or `Cmd+M` followed by `R` (Mac)
- In VS Code: `Ctrl+Shift+P` (Windows/Linux) or `Cmd+Shift+P` (Mac) to open the command palette, then type 'restart' and select 'Restart Kernel'

After restarting, run the cells again from the beginning.

## 1. Setup and Library Imports

In [13]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import zipfile
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

# Define a simple progress bar replacement instead of using tqdm
def simple_progress_bar(iterable=None, total=None, desc="Progress"):
    """A very simple progress bar replacement for tqdm"""
    if iterable is not None:
        total = len(iterable)
        it = iter(iterable)
    else:
        it = range(total)
    
    def progress_iterator():
        print(f"{desc}: 0/{total} (0%)")
        count = 0
        for item in it:
            yield item
            count += 1
            if count % max(1, total // 20) == 0 or count == total:  # Update ~20 times total
                percent = int(100 * count / total)
                print(f"\r{desc}: {count}/{total} ({percent}%)", end="")
        print()  # New line at the end
        
    return progress_iterator() if iterable is not None else progress_iterator

# Replace tqdm with our simple progress bar
tqdm = simple_progress_bar
trange = lambda *args, **kwargs: simple_progress_bar(range(*args), **kwargs)

# Set plot styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# Configure pandas display options
pd.set_option('display.max_columns', None)

# Random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## 2. Define Run Identifier and Load Data

Before loading the data, please specify a unique identifier for this training run. This identifier will be used to create separate directories for the models, results, and visualizations, helping to organize outputs from different datasets or experiments.

For example, if you are processing a dataset named 'dataset_alpha_v1', you could set `run_identifier = "dataset_alpha_v1"`.

In [ ]:
# DEFINE YOUR RUN IDENTIFIER HERE
# This name will be used to create specific subdirectories for this run's outputs.
run_identifier = "df_base_initial_run"  # Example: "series_A_experiment_1"

if not run_identifier or not isinstance(run_identifier, str) or "\" in run_identifier or "/" in run_identifier:
    raise ValueError("Please set a valid 'run_identifier' string (e.g., 'my_dataset_run_1'). It should not contain path separators.")

print(f"Using run identifier: {run_identifier}")

## 2. Data Loading and Exploration

We'll load the data from either a CSV file or extract it from a ZIP archive if needed.

In [ ]:
# Create timestamp for versioning
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Set up directories based on run_identifier
base_models_dir = f'models/{run_identifier}'
base_results_dir = f'results/{run_identifier}'

models_dir = f'{base_models_dir}/xgboost_{timestamp}'
results_dir = f'{base_results_dir}/xgboost_{timestamp}'
data_dir = 'data' # Data directory remains the same

# Create directories if they don't exist
for directory in [data_dir, models_dir, results_dir]:
    os.makedirs(directory, exist_ok=True)

print(f"Run Identifier: {run_identifier}")
print(f"Models will be saved in: {models_dir}")
print(f"Results will be saved in: {results_dir}")

# Function to load data from CSV or extract from ZIP
def load_data():
    csv_path = os.path.join(data_dir, 'df_base.csv')
    zip_path = os.path.join(data_dir, 'df_base.zip')
    
    if os.path.exists(csv_path):
        print(f"Loading data from {csv_path}...")
        df = pd.read_csv(csv_path)
    elif os.path.exists(zip_path):
        print(f"Extracting data from {zip_path}...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        df = pd.read_csv(csv_path)
    else:
        raise FileNotFoundError(f"Neither {csv_path} nor {zip_path} found. Please check the data location.")
    
    return df

# Load the data
try:
    df_base = load_data()
    print(f"Data loaded successfully with {df_base.shape[0]} rows and {df_base.shape[1]} columns.")
except Exception as e:
    print(f"Error loading data: {e}")
    # Create a dummy dataset for demonstration if needed
    print("Creating dummy dataset for demonstration...")
    # This is just for demonstration - adjust based on your actual data structure
    n_municipalities = 5
    weeks_per_mun = 104  # 2 years of weekly data
    mun_codes = [f"{i:05d}" for i in range(1, n_municipalities + 1)]
    df_base = pd.DataFrame({
        'CD_MUN': np.repeat(mun_codes, weeks_per_mun),
        'week': np.tile(np.arange(weeks_per_mun), n_municipalities),
        'target': np.random.normal(50, 10, n_municipalities * weeks_per_mun)
    })

Loading data from data\df_base.csv...
Data loaded successfully with 6684000 rows and 11 columns.


In [5]:
# Basic data exploration
print("Data overview:")
display(df_base.head())

# Check data types
print("\nData types:")
display(df_base.dtypes)

# Convert CD_MUN to string if not already
df_base["CD_MUN"] = df_base["CD_MUN"].astype(str)

# Summary statistics
print("\nSummary statistics:")
display(df_base.describe())

# Check for missing values
print("\nMissing values:")
display(df_base.isnull().sum())

# Count municipalities
n_municipalities = df_base["CD_MUN"].nunique()
print(f"\nNumber of unique municipalities: {n_municipalities}")

# Check data distribution by municipality
mun_counts = df_base["CD_MUN"].value_counts()
print(f"\nData distribution by municipality:")
display(mun_counts.describe())

Data overview:


,CD_MUN,week,target,PIB,DENS,URB,CO2,CH4,N2O,LAT,LON
0,1100015,0,0.515856,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39
1,1100015,1,0.539765,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39
2,1100015,2,0.458823,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39
3,1100015,3,0.485555,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39
4,1100015,4,0.231805,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39



Data types:


CD_MUN      int64
week        int64
target    float64
PIB       float64
DENS      float64
URB       float64
CO2       float64
CH4       float64
N2O       float64
LAT       float64
LON       float64
dtype: object


Summary statistics:


,week,target,PIB,DENS,URB,CO2,CH4,N2O,LAT,LON
count,6.684000e+06,6.684000e+06,6.003816e+06,6.033600e+06,6.033600e+06,6.033600e+06,6.033600e+06,6.033600e+06,6.033600e+06,6.033600e+06
mean,5.995000e+02,4.710729e-01,1.397299e+04,8.091379e+01,1.268524e-02,3.953797e+04,7.463403e+02,1.116126e+01,-1.638302e+01,-4.637996e+01
std,3.464101e+02,3.616928e+00,1.973538e+04,3.952444e+02,4.668693e-02,1.244388e+05,1.332048e+04,2.845796e+01,8.354524e+00,6.424812e+00
min,0.000000e+00,0.000000e+00,-1.459830e+03,3.143629e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.365254e+01,-7.348400e+01
25%,2.997500e+02,0.000000e+00,4.516650e+03,1.112562e+01,1.319166e-03,4.454221e+03,6.922971e+01,4.712564e+00,-2.279264e+01,-5.102275e+01
50%,5.995000e+02,0.000000e+00,8.607535e+03,2.363456e+01,2.999792e-03,1.128696e+04,1.379897e+02,8.660524e+00,-1.785337e+01,-4.661550e+01
75%,8.992500e+02,2.773210e-01,1.684539e+04,4.817254e+01,7.148583e-03,2.817704e+04,2.405503e+02,1.352129e+01,-8.456431e+00,-4.166000e+01
max,1.199000e+03,7.359496e+02,9.208340e+05,1.344249e+04,9.768962e-01,3.137265e+06,7.075641e+05,3.683565e+03,4.685425e+00,-3.487000e+01



Missing values:


CD_MUN         0
week           0
target         0
PIB       680184
DENS      650400
URB       650400
CO2       650400
CH4       650400
N2O       650400
LAT       650400
LON       650400
dtype: int64


Number of unique municipalities: 5570

Data distribution by municipality:


count    5570.0
mean     1200.0
std         0.0
min      1200.0
25%      1200.0
50%      1200.0
75%      1200.0
max      1200.0
Name: count, dtype: float64

## 3. Data Preprocessing

Here we'll prepare the data for time series modeling. This includes normalization and handling date/time information.

In [6]:
# Normalize target values per municipality
print("Normalizing target values per municipality...")
df_base["target_norm"] = df_base.groupby("CD_MUN")["target"].transform(
    lambda x: (x - x.mean()) / (x.std() + 1e-8)
)

# Ensure data is ordered by municipality and week
df_base = df_base.sort_values(["CD_MUN", "week"])

# Check if we have any date columns; if not, create a date feature
if not any(col.lower() in ['date', 'datetime', 'time'] for col in df_base.columns):
    print("Creating date features from week column...")
    # Assuming week is a sequential counter (0, 1, 2, ...) for each municipality
    df_base['date'] = pd.to_datetime('2018-01-01') + \
                     pd.to_timedelta(df_base['week'] * 7, unit='D')
    
    # Extract date components
    df_base['year'] = df_base['date'].dt.year
    df_base['month'] = df_base['date'].dt.month
    df_base['day_of_week'] = df_base['date'].dt.dayofweek
    df_base['day_of_year'] = df_base['date'].dt.dayofyear
    df_base['week_of_year'] = df_base['date'].dt.isocalendar().week
    
print("Data preprocessing completed.")
display(df_base.head())

Normalizing target values per municipality...
Creating date features from week column...
Data preprocessing completed.


,CD_MUN,week,target,PIB,DENS,URB,CO2,CH4,N2O,LAT,LON,target_norm,date,year,month,day_of_week,day_of_year,week_of_year
0,1100015,0,0.515856,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39,0.985625,2018-01-01,2018,1,0,1,1
1,1100015,1,0.539765,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39,1.075675,2018-01-08,2018,1,0,8,2
2,1100015,2,0.458823,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39,0.770817,2018-01-15,2018,1,0,15,3
3,1100015,3,0.485555,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39,0.871500,2018-01-22,2018,1,0,22,4
4,1100015,4,0.231805,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39,-0.084223,2018-01-29,2018,1,0,29,5


## 4. Feature Engineering

Here we'll create features specific to time series modeling, focusing on implementing a sliding window approach.

In [7]:
# Define feature engineering functions
def create_lag_features(df, target_col, lag_list, group_col="CD_MUN"):
    """Create lag features for a grouped time series."""
    df_copy = df.copy()
    
    for lag in lag_list:
        lag_col = f"{target_col}_lag_{lag}"
        df_copy[lag_col] = df_copy.groupby(group_col)[target_col].shift(lag)
        
    return df_copy

def create_rolling_features(df, target_col, windows, group_col="CD_MUN"):
    """Create rolling window features (mean, std, min, max) for a grouped time series."""
    df_copy = df.copy()
    
    for window in windows:
        # Rolling mean
        df_copy[f"{target_col}_rolling_mean_{window}"] = (
            df_copy.groupby(group_col)[target_col]
            .rolling(window=window, min_periods=1)
            .mean()
            .reset_index(level=0, drop=True)
        )
        
        # Rolling std
        df_copy[f"{target_col}_rolling_std_{window}"] = (
            df_copy.groupby(group_col)[target_col]
            .rolling(window=window, min_periods=1)
            .std()
            .reset_index(level=0, drop=True)
        )
        
        # Rolling min/max
        df_copy[f"{target_col}_rolling_min_{window}"] = (
            df_copy.groupby(group_col)[target_col]
            .rolling(window=window, min_periods=1)
            .min()
            .reset_index(level=0, drop=True)
        )
        
        df_copy[f"{target_col}_rolling_max_{window}"] = (
            df_copy.groupby(group_col)[target_col]
            .rolling(window=window, min_periods=1)
            .max()
            .reset_index(level=0, drop=True)
        )
    
    return df_copy

def create_target_features(df, target_col, forecast_periods=4, group_col="CD_MUN"):
    """Create target features (sum of next n periods) for a grouped time series."""
    df_copy = df.copy()
    
    # Create the target: sum of next 4 weeks
    target_values = []
    
    for _, group in df_copy.groupby(group_col):
        group_values = group[target_col].values
        target_array = np.zeros(len(group_values))
        
        for i in range(len(group_values) - forecast_periods):
            target_array[i] = np.sum(group_values[i+1:i+1+forecast_periods])
            
        target_values.append(target_array)
    
    df_copy[f"{target_col}_next_{forecast_periods}_sum"] = np.concatenate(target_values)
    
    # Create the normalized target (using group stats)
    df_copy[f"{target_col}_next_{forecast_periods}_sum_norm"] = (
        df_copy.groupby(group_col)[f"{target_col}_next_{forecast_periods}_sum"]
        .transform(lambda x: (x - x.mean()) / (x.std() + 1e-8))
    )
    
    return df_copy

# Apply feature engineering
print("Creating lag features...")
lags = [1, 2, 3, 4, 8, 12, 24, 48]  # Weekly lags
df_features = create_lag_features(df_base, "target_norm", lags)

print("Creating rolling window features...")
windows = [4, 8, 12, 24, 48]  # Weekly rolling windows
df_features = create_rolling_features(df_features, "target_norm", windows)

print("Creating target features...")
df_features = create_target_features(df_features, "target", forecast_periods=4)

print("Feature engineering completed.")
print(f"Total features created: {df_features.shape[1] - df_base.shape[1]}")

# Display the resulting dataframe
display(df_features.head())

Creating lag features...
Creating rolling window features...
Creating target features...
Feature engineering completed.
Total features created: 30


,CD_MUN,week,target,PIB,DENS,URB,CO2,CH4,N2O,LAT,LON,target_norm,date,year,month,day_of_week,day_of_year,week_of_year,target_norm_lag_1,target_norm_lag_2,target_norm_lag_3,target_norm_lag_4,target_norm_lag_8,target_norm_lag_12,target_norm_lag_24,target_norm_lag_48,target_norm_rolling_mean_4,target_norm_rolling_std_4,target_norm_rolling_min_4,target_norm_rolling_max_4,target_norm_rolling_mean_8,target_norm_rolling_std_8,target_norm_rolling_min_8,target_norm_rolling_max_8,target_norm_rolling_mean_12,target_norm_rolling_std_12,target_norm_rolling_min_12,target_norm_rolling_max_12,target_norm_rolling_mean_24,target_norm_rolling_std_24,target_norm_rolling_min_24,target_norm_rolling_max_24,target_norm_rolling_mean_48,target_norm_rolling_std_48,target_norm_rolling_min_48,target_norm_rolling_max_48,target_next_4_sum,target_next_4_sum_norm
0,1100015,0,0.515856,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39,0.985625,2018-01-01,2018,1,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.985625,NaN,0.985625,0.985625,0.985625,NaN,0.985625,0.985625,0.985625,NaN,0.985625,0.985625,0.985625,NaN,0.985625,0.985625,0.985625,NaN,0.985625,0.985625,1.715949,0.774298
1,1100015,1,0.539765,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39,1.075675,2018-01-08,2018,1,0,8,2,0.985625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.030650,0.063675,0.985625,1.075675,1.030650,0.063675,0.985625,1.075675,1.030650,0.063675,0.985625,1.075675,1.030650,0.063675,0.985625,1.075675,1.030650,0.063675,0.985625,1.075675,1.409381,0.438793
2,1100015,2,0.458823,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39,0.770817,2018-01-15,2018,1,0,15,3,1.075675,0.985625,NaN,NaN,NaN,NaN,NaN,NaN,0.944039,0.156626,0.770817,1.075675,0.944039,0.156626,0.770817,1.075675,0.944039,0.156626,0.770817,1.075675,0.944039,0.156626,0.770817,1.075675,0.944039,0.156626,0.770817,1.075675,1.226187,0.238308
3,1100015,3,0.485555,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39,0.871500,2018-01-22,2018,1,0,22,4,0.770817,1.075675,0.985625,NaN,NaN,NaN,NaN,NaN,0.925904,0.132928,0.770817,1.075675,0.925904,0.132928,0.770817,1.075675,0.925904,0.132928,0.770817,1.075675,0.925904,0.132928,0.770817,1.075675,0.925904,0.132928,0.770817,1.075675,1.000000,-0.009228
4,1100015,4,0.231805,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39,-0.084223,2018-01-29,2018,1,0,29,5,0.871500,0.770817,1.075675,0.985625,NaN,NaN,NaN,NaN,0.658442,0.511096,-0.084223,1.075675,0.723879,0.466180,-0.084223,1.075675,0.723879,0.466180,-0.084223,1.075675,0.723879,0.466180,-0.084223,1.075675,0.723879,0.466180,-0.084223,1.075675,1.286916,0.304770


## 5. Training Framework

Here we'll set up the framework for training the XGBoost models, including time series splitting and model configuration.

In [8]:
# Define constants
LOOKBACK_PERIOD = 48  # 48 weeks lookback
FORECAST_HORIZON = 4  # Predict sum of next 4 weeks
TEST_SIZE = 12  # Use last 12 weeks for testing

# Define time series split function
def time_series_split(df, test_size=TEST_SIZE, min_train_size=LOOKBACK_PERIOD+FORECAST_HORIZON, group_col="CD_MUN"):
    """Split a dataframe into training and testing sets by municipality, preserving time order."""
    train_dfs = []
    test_dfs = []
    skipped_munis = []
    
    for mun, group in df.groupby(group_col):
        if len(group) < min_train_size + test_size:
            skipped_munis.append(mun)
            continue
            
        # Split by index (preserving time order)
        split_idx = len(group) - test_size
        train_dfs.append(group.iloc[:split_idx])
        test_dfs.append(group.iloc[split_idx:])
    
    if skipped_munis:
        print(f"Skipped {len(skipped_munis)} municipalities with insufficient data")
    
    if not train_dfs or not test_dfs:
        raise ValueError("No valid municipalities found with sufficient data.")
        
    return pd.concat(train_dfs), pd.concat(test_dfs), skipped_munis

# Function to prepare X, y for a single municipality
def prepare_xy_data(df, target_col, feature_cols):
    """Prepare X and y data, removing rows with NaN from feature engineering."""
    df_valid = df.dropna(subset=feature_cols + [target_col])
    X = df_valid[feature_cols].values
    y = df_valid[target_col].values
    return X, y, df_valid

# Split data into train and test sets
try:
    # Drop NaN values from feature engineering
    df_features_clean = df_features.dropna()
    
    # Define feature columns (excluding target columns and metadata)
    exclude_cols = [
        'target_next_4_sum', 'target_next_4_sum_norm', 
        'CD_MUN', 'week', 'target', 'date'
    ]
    feature_cols = [col for col in df_features_clean.columns 
                   if col not in exclude_cols]
    
    # Perform the split
    df_train, df_test, skipped_munis = time_series_split(df_features_clean)
    print(f"Data split completed. Train shape: {df_train.shape}, Test shape: {df_test.shape}")
    print(f"Number of features: {len(feature_cols)}")
except Exception as e:
    print(f"Error in data splitting: {e}")
    # Create placeholder dataframes in case of error
    df_train = df_features_clean.iloc[:int(0.8*len(df_features_clean))]
    df_test = df_features_clean.iloc[int(0.8*len(df_features_clean)):]
    feature_cols = [col for col in df_features_clean.columns 
                   if col not in ['target_next_4_sum', 'target_next_4_sum_norm', 
                                 'CD_MUN', 'week', 'target']]
    skipped_munis = []

Data split completed. Train shape: (5704872, 48), Test shape: (60336, 48)
Number of features: 42


## 6. Model Training Loop

Now we'll train one XGBoost model per municipality using the prepared features.

In [14]:
# Configure XGBoost parameters
xgb_params = {
    'n_estimators': 100,
    'learning_rate': 0.1,
    'max_depth': 5,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'random_state': RANDOM_SEED,
    'objective': 'reg:squarederror',
    'n_jobs': -1
}

# Function to train a model for a single municipality
def train_municipality_model(mun_code, train_df, test_df, feature_cols,
                            target_col='target_next_4_sum_norm',
                            params=xgb_params):
    """Train and evaluate XGBoost model for a single municipality."""
    # Filter data for this municipality
    mun_train = train_df[train_df['CD_MUN'] == mun_code]
    mun_test = test_df[test_df['CD_MUN'] == mun_code]
    
    # Prepare features and target
    X_train, y_train, valid_train = prepare_xy_data(mun_train, target_col, feature_cols)
    X_test, y_test, valid_test = prepare_xy_data(mun_test, target_col, feature_cols)
    
    # Skip if insufficient data
    if len(X_train) < 10 or len(X_test) < 2:
        return None, None, None, None, f"Insufficient data for municipality {mun_code}"
    
    # Train XGBoost model
    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Save results for evaluation
    train_results = valid_train.copy()
    train_results['prediction_norm'] = y_pred_train
    
    test_results = valid_test.copy()
    test_results['prediction_norm'] = y_pred_test
    
    # Denormalize predictions
    # Get statistics from the original dataframe for denormalization
    mun_stats = df_features[df_features['CD_MUN'] == mun_code]['target_next_4_sum']
    target_mean = mun_stats.mean()
    target_std = mun_stats.std()
    
    # Denormalize predictions
    train_results['prediction'] = y_pred_train * target_std + target_mean
    test_results['prediction'] = y_pred_test * target_std + target_mean
    
    return model, train_results, test_results, feature_cols, None

# Function to train models for all municipalities
def train_all_municipalities(train_df, test_df, feature_cols, model_dir):
    """Train XGBoost models for all municipalities in the dataset."""
    all_municipalities = sorted(train_df['CD_MUN'].unique())
    
    results = {
        'models': {},
        'train_results': {},
        'test_results': {},
        'feature_cols': feature_cols,
        'errors': {}
    }
    
    print(f"Training models for {len(all_municipalities)} municipalities...")
    
    # Use manual counter instead of tqdm
    total_munis = len(all_municipalities)
    print(f"0/{total_munis} (0%)")
    
    for i, mun_code in enumerate(all_municipalities):
        try:
            model, train_res, test_res, feat_cols, error = train_municipality_model(
                mun_code, train_df, test_df, feature_cols)
            
            if error:
                results['errors'][mun_code] = error
                continue
                
            # Save results
            results['models'][mun_code] = model
            results['train_results'][mun_code] = train_res
            results['test_results'][mun_code] = test_res
            
            # Save model to disk
            mun_dir = os.path.join(model_dir, mun_code)
            os.makedirs(mun_dir, exist_ok=True)
            joblib.dump(model, os.path.join(mun_dir, 'model.pkl'))
            
            # Print progress every 50 items or at the end
            if (i + 1) % 50 == 0 or i + 1 == total_munis:
                percent = int(100 * (i + 1) / total_munis)
                print(f"\r{i + 1}/{total_munis} ({percent}%)", end="")
                
        except Exception as e:
            results['errors'][mun_code] = str(e)
            print(f"\nError training model for municipality {mun_code}: {e}")
    
    print()  # Add a new line after the progress display
    print(f"Training completed. {len(results['models'])} models trained successfully.")
    print(f"Errors occurred for {len(results['errors'])} municipalities.")
    
    return results

# Train all models
try:
    training_results = train_all_municipalities(df_train, df_test, feature_cols, models_dir)
    
    # Save training results
    results_file = os.path.join(results_dir, 'training_results.pkl')
    joblib.dump(training_results, results_file)
    print(f"Training results saved to {results_file}")
except Exception as e:
    print(f"Error in model training: {e}")

Training models for 5028 municipalities...
0/5028 (0%)
5028/5028 (100%)
Training completed. 5028 models trained successfully.
Errors occurred for 0 municipalities.
Training results saved to results/xgboost_20250505_115446\training_results.pkl


## 7. Evaluation and Visualization

Now we'll evaluate the models and create comprehensive visualizations.

In [15]:
# Function to calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    """Calculate regression metrics."""
    metrics = {
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R2': r2_score(y_true, y_pred)
    }
    return metrics

# Function to create evaluation report
def create_evaluation_report(training_results):
    """Create evaluation report for all municipalities."""
    metrics = {
        'normalized': {'train': {}, 'test': {}},
        'denormalized': {'train': {}, 'test': {}}
    }
    
    for mun_code, train_res in training_results['train_results'].items():
        test_res = training_results['test_results'][mun_code]
        
        # Calculate normalized metrics
        metrics['normalized']['train'][mun_code] = calculate_metrics(
            train_res['target_next_4_sum_norm'], train_res['prediction_norm'])
        metrics['normalized']['test'][mun_code] = calculate_metrics(
            test_res['target_next_4_sum_norm'], test_res['prediction_norm'])
        
        # Calculate denormalized metrics
        metrics['denormalized']['train'][mun_code] = calculate_metrics(
            train_res['target_next_4_sum'], train_res['prediction'])
        metrics['denormalized']['test'][mun_code] = calculate_metrics(
            test_res['target_next_4_sum'], test_res['prediction'])
    
    return metrics

# Calculate metrics
try:
    evaluation_metrics = create_evaluation_report(training_results)
    
    # Save metrics
    metrics_file = os.path.join(results_dir, 'evaluation_metrics.pkl')
    joblib.dump(evaluation_metrics, metrics_file)
    print(f"Evaluation metrics saved to {metrics_file}")
    
    # Display aggregate metrics
    print("\nAggregate Test Metrics (Normalized):")
    norm_test_metrics = pd.DataFrame(evaluation_metrics['normalized']['test']).T
    display(norm_test_metrics.describe())
    
    print("\nAggregate Test Metrics (Denormalized):")
    denorm_test_metrics = pd.DataFrame(evaluation_metrics['denormalized']['test']).T
    display(denorm_test_metrics.describe())
except Exception as e:
    print(f"Error calculating metrics: {e}")

Evaluation metrics saved to results/xgboost_20250505_115446\evaluation_metrics.pkl

Aggregate Test Metrics (Normalized):


,MAE,RMSE,R2
count,5028.000000,5028.000000,5.028000e+03
mean,0.912692,1.088112,-2.062272e+31
std,0.476112,0.547626,1.439847e+32
min,0.012371,0.014906,-3.586719e+33
25%,0.590019,0.716323,-2.073606e+00
50%,0.826360,0.991476,-5.304416e-01
75%,1.131942,1.348222,0.000000e+00
max,4.651602,4.843684,9.847828e-01



Aggregate Test Metrics (Denormalized):


,MAE,RMSE,R2
count,5028.000000,5028.000000,5028.000000
mean,0.785864,0.938022,-7.889956
std,1.298722,1.487579,138.031595
min,0.002904,0.002934,-7200.657771
25%,0.348114,0.412969,-1.358671
50%,0.545706,0.657979,-0.318405
75%,0.888948,1.063561,0.000000
max,63.616640,70.368439,0.984783


In [ ]:
# Function to create visualizations
def create_visualizations(training_results, metrics, results_dir):
    """Create comprehensive visualizations for model evaluation."""
    # viz_dir is now a subdirectory of the main results_dir for this run
    viz_dir = os.path.join(results_dir, 'visualizations')
    os.makedirs(viz_dir, exist_ok=True)
    
    try:
        print("Creating error distribution plot...")
        # 1. Error Distribution Plot (overall)
        plt.figure(figsize=(12, 6))
        
        all_errors = []
        for mun_code, test_res in training_results['test_results'].items():
            errors = test_res['target_next_4_sum'] - test_res['prediction']
            all_errors.extend(errors)
        
        sns.histplot(all_errors, kde=True)
        plt.title('Error Distribution Across All Municipalities')
        plt.xlabel('Error (Actual - Predicted)')
        plt.savefig(os.path.join(viz_dir, 'error_distribution_overall.png'))
        plt.close()
        
        print("Creating RMSE comparison plot...")
        # 2. Performance Comparison Across Municipalities
        try:
            metrics_df = pd.DataFrame({
                mun: metrics['denormalized']['test'][mun]['RMSE'] 
                for mun in list(metrics['denormalized']['test'].keys())
            }, index=['RMSE']).T
            
            metrics_df = metrics_df.sort_values('RMSE')
            
            plt.figure(figsize=(14, 8))
            sns.barplot(x=metrics_df.index, y='RMSE', data=metrics_df)
            plt.title('RMSE by Municipality')
            plt.xticks(rotation=90)
            plt.tight_layout()
            plt.savefig(os.path.join(viz_dir, 'rmse_by_municipality.png'))
            plt.close()
        except Exception as e:
            print(f"Error creating RMSE comparison plot: {e}")
        
        # 3. Individual municipality visualizations (sample based on performance)
        print("Creating per-municipality visualizations (Top 100 by R2 + São Paulo)...")
        
        # Get top 100 municipalities by R2 score (denormalized, test set)
        try:
            denorm_test_metrics = pd.DataFrame(metrics['denormalized']['test']).T.sort_values('R2', ascending=False)
            top_munis = denorm_test_metrics.head(100).index.tolist()
            
            # Ensure São Paulo is included
            sao_paulo_code = '3550308'
            if sao_paulo_code in training_results['models'] and sao_paulo_code not in top_munis:
                if len(top_munis) >= 100:
                    top_munis.pop() # Remove the last one to make space if needed
                top_munis.append(sao_paulo_code)
            
            sample_munis = top_munis
            print(f"Selected {len(sample_munis)} municipalities for detailed plots.")

        except Exception as e:
            print(f"Error selecting top municipalities, falling back to first 5: {e}")
            sample_munis = list(training_results['models'].keys())[:5]

        # Limit to available models just in case
        sample_munis = [m for m in sample_munis if m in training_results['models']]
        
        # Use a simpler progress indicator for plotting
        total_plot_munis = len(sample_munis)
        print(f"Generating plots for {total_plot_munis} municipalities...")
        plot_count = 0
        update_interval = max(1, total_plot_munis // 10) # Update roughly 10 times

        for mun_code in sample_munis:
            print(f"Processing municipality {mun_code}...")
            mun_dir = os.path.join(viz_dir, mun_code)
            os.makedirs(mun_dir, exist_ok=True)
            
            train_res = training_results['train_results'][mun_code]
            test_res = training_results['test_results'][mun_code]
            model = training_results['models'][mun_code]
            
            # 3.1 Actual vs Predicted
            plt.figure(figsize=(10, 6))
            plt.scatter(test_res['target_next_4_sum'], test_res['prediction'], alpha=0.7)
            plt.plot([test_res['target_next_4_sum'].min(), test_res['target_next_4_sum'].max()], 
                    [test_res['target_next_4_sum'].min(), test_res['target_next_4_sum'].max()], 
                    'r--')
            plt.xlabel('Actual')
            plt.ylabel('Predicted')
            plt.title(f'Actual vs Predicted - Municipality {mun_code}')
            plt.savefig(os.path.join(mun_dir, 'actual_vs_predicted.png'))
            plt.close()
            
            # 3.2 Time Series Forecast
            plt.figure(figsize=(12, 6))
            # Use 'date' column for x-axis
            plt.plot(test_res['date'], test_res['target_next_4_sum'], 'b-', label='Actual')
            plt.plot(test_res['date'], test_res['prediction'], 'r--', label='Predicted')
            plt.title(f'Time Series Forecast - Municipality {mun_code}')
            plt.xlabel('Date') # Changed label
            plt.ylabel('Target (Sum of Next 4 Weeks)')
            plt.legend()
            plt.grid(True)
            plt.xticks(rotation=45) # Rotate date labels for better readability
            plt.tight_layout() # Adjust layout
            plt.savefig(os.path.join(mun_dir, 'time_series_forecast.png'))
            plt.close()
            
            # 3.3 Feature Importance
            try:
                feature_importance = model.feature_importances_
                feature_names = training_results['feature_cols']
                
                importance_df = pd.DataFrame({
                    'Feature': feature_names,
                    'Importance': feature_importance
                }).sort_values('Importance', ascending=False).head(15)
                
                plt.figure(figsize=(12, 8))
                sns.barplot(x='Importance', y='Feature', data=importance_df)
                plt.title(f'Feature Importance - Municipality {mun_code}')
                plt.tight_layout()
                plt.savefig(os.path.join(mun_dir, 'feature_importance.png'))
                plt.close()
            except Exception as e:
                print(f"Error creating feature importance plot for {mun_code}: {e}")
            
            # 3.4 Residual Analysis
            residuals = test_res['target_next_4_sum'] - test_res['prediction']
            
            plt.figure(figsize=(12, 6))
            plt.scatter(test_res['prediction'], residuals)
            plt.axhline(y=0, color='r', linestyle='--')
            plt.title(f'Residual Analysis - Municipality {mun_code}')
            plt.xlabel('Predicted Values')
            plt.ylabel('Residuals')
            plt.grid(True)
            plt.savefig(os.path.join(mun_dir, 'residual_analysis.png'))
            plt.close()
            
            # Update progress
            plot_count += 1
            if plot_count % update_interval == 0 or plot_count == total_plot_munis:
                percent_done = int(100 * plot_count / total_plot_munis)
                print(f"\rPlotting progress: {plot_count}/{total_plot_munis} ({percent_done}%)", end="")
        
        print("\nFinished per-municipality plots.") # Newline after progress

        print(f"Visualizations saved to {viz_dir}")
        return viz_dir
    except Exception as e:
        import traceback
        print(f"Error creating visualizations: {e}")
        print(traceback.format_exc())
        return viz_dir

# Create visualizations
try:
    viz_path = create_visualizations(training_results, evaluation_metrics, results_dir)
    print(f"Visualizations created successfully in {viz_path}")
except Exception as e:
    print(f"Error creating visualizations: {e}")

Creating error distribution plot...
Creating RMSE comparison plot...
Creating RMSE comparison plot...
Creating per-municipality visualizations (Top 100 by R2 + São Paulo)...
Selected 100 municipalities for detailed plots.
Generating plots for 100 municipalities...
Processing municipality 3101300...
Creating per-municipality visualizations (Top 100 by R2 + São Paulo)...
Selected 100 municipalities for detailed plots.
Generating plots for 100 municipalities...
Processing municipality 3101300...
Processing municipality 4305157...
Processing municipality 4305157...
Processing municipality 3160900...
Processing municipality 3160900...
Processing municipality 4106555...
Processing municipality 4106555...
Processing municipality 4323705...
Processing municipality 4323705...
Processing municipality 2706505...
Processing municipality 2706505...
Processing municipality 1708254...
Processing municipality 1708254...
Processing municipality 1702901...
Processing municipality 1702901...
Processing m

## 8. Results Summary

Let's summarize the results and create a final report.

In [17]:
# Create a summary report
def create_summary_report(training_results, metrics, results_dir):
    """Create a comprehensive summary report of the modeling results."""
    # Count successful models
    successful_models = len(training_results['models'])
    failed_models = len(training_results['errors'])
    total_municipalities = successful_models + failed_models + len(skipped_munis)
    
    # Calculate average metrics
    norm_test_metrics = pd.DataFrame(metrics['normalized']['test']).T
    denorm_test_metrics = pd.DataFrame(metrics['denormalized']['test']).T
    
    # Create report dictionary
    report = {
        'timestamp': timestamp,
        'model_counts': {
            'successful': successful_models,
            'failed': failed_models,
            'skipped': len(skipped_munis),
            'total': total_municipalities
        },
        'average_metrics': {
            'normalized': norm_test_metrics.mean().to_dict(),
            'denormalized': denorm_test_metrics.mean().to_dict()
        },
        'best_municipalities': {
            'by_rmse': denorm_test_metrics.sort_values('RMSE').iloc[:5].index.tolist(),
            'by_r2': denorm_test_metrics.sort_values('R2', ascending=False).iloc[:5].index.tolist()
        },
        'worst_municipalities': {
            'by_rmse': denorm_test_metrics.sort_values('RMSE', ascending=False).iloc[:5].index.tolist(),
            'by_r2': denorm_test_metrics.sort_values('R2').iloc[:5].index.tolist()
        },
        'paths': {
            'models': models_dir,
            'results': results_dir
        }
    }
    
    # Save report to JSON
    report_file = os.path.join(results_dir, 'summary_report.json')
    with open(report_file, 'w') as f:
        import json
        json.dump(report, f, indent=2)
    
    return report

# Create and display summary report
try:
    summary = create_summary_report(training_results, evaluation_metrics, results_dir)
    print("\n=== XGBoost Training Summary ===")
    print(f"Timestamp: {summary['timestamp']}")
    print(f"Total municipalities: {summary['model_counts']['total']}")
    print(f"Models successfully trained: {summary['model_counts']['successful']}")
    print(f"Models failed: {summary['model_counts']['failed']}")
    print(f"Municipalities skipped: {summary['model_counts']['skipped']}")
    print("\nAverage Test Metrics (Denormalized):")
    for metric, value in summary['average_metrics']['denormalized'].items():
        print(f"  {metric}: {value:.4f}")
    print("\nBest municipalities by R²:")
    for i, mun in enumerate(summary['best_municipalities']['by_r2']):
        r2 = evaluation_metrics['denormalized']['test'][mun]['R2']
        print(f"  {i+1}. {mun}: R² = {r2:.4f}")
    print("\nResults saved to:")
    print(f"  Models: {summary['paths']['models']}")
    print(f"  Results: {summary['paths']['results']}")
except Exception as e:
    print(f"Error creating summary report: {e}")


=== XGBoost Training Summary ===
Timestamp: 20250505_115446
Total municipalities: 5028
Models successfully trained: 5028
Models failed: 0
Municipalities skipped: 0

Average Test Metrics (Denormalized):
  MAE: 0.7859
  RMSE: 0.9380
  R2: -7.8900

Best municipalities by R²:
  1. 3101300: R² = 0.9848
  2. 4305157: R² = 0.9318
  3. 3160900: R² = 0.9131
  4. 4106555: R² = 0.9069
  5. 4323705: R² = 0.8954

Results saved to:
  Models: models/xgboost_20250505_115446
  Results: results/xgboost_20250505_115446
